In [1]:
import os

import matplotlib.pyplot as plt
import mne
import numpy as np
import pandas as pd
import pingouin as pg
import seaborn as sns
import xarray as xr
from plotting import plot_eeg_topomaps
from scipy.stats import zscore
from tqdm import tqdm
from utils import run_in_parallel

plt.style.use("papers")

DATA_FOLDER = "../../data/test_data_LEMON/"
RESULTS_FOLDER = "/Users/nikola/work-brain/eeg_microstate_and_latent_space/results/20220728_VARprocess_EC_allsubjects_30.0s_segment_1200.0s_VARlength"

In [2]:
topomaps = xr.open_dataarray(os.path.join(RESULTS_FOLDER, "topomaps.nc"))
topomaps = topomaps.set_index({"stack": ["subject_id", "type"]}).unstack()

In [15]:
d.dropna("channels")

<xarray.DataArray (channels: 59, type: 43)>
array([[ 0.01508535, -0.09423991,  0.00801241, ..., -0.03025483,
        -0.08038782, -0.0699523 ],
       [ 0.06112219, -0.11715402,  0.0448409 , ..., -0.06086882,
        -0.10992959, -0.1033738 ],
       [-0.00942609, -0.05322346, -0.00406897, ...,  0.03713127,
        -0.04745408, -0.01622444],
       ...,
       [-0.05667301,  0.16205941, -0.04491549, ...,  0.10637297,
         0.1228966 ,  0.13704982],
       [ 0.08004519,  0.05428901,  0.09666041, ..., -0.01329099,
         0.00238484,  0.03451508],
       [ 0.13778789,  0.03369196,  0.17794618, ..., -0.0006103 ,
        -0.05670019,  0.03502629]])
Coordinates:
    latent map  <U1 'A'
  * channels    (channels) int64 0 1 2 3 4 5 6 7 8 ... 51 52 53 54 55 56 57 58
    subject_id  <U10 'sub-010002'
  * type        (type) object '1st-segment' '2nd-segment' ... 'VAR-full'

In [67]:
def corr2_coeff(A, B):
    # Rowwise mean of input arrays & subtract from input arrays themeselves
    A_mA = A - A.mean(1)[:, None]
    B_mB = B - B.mean(1)[:, None]

    # Sum of squares across rows
    ssA = (A_mA**2).sum(1)
    ssB = (B_mB**2).sum(1)

    # Finally get corr coeff
    return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None], ssB[None]))

In [68]:
xrs = []
pbar = tqdm(
    total=len(topomaps["latent map"])
    * len(list(combinations_with_replacement(topomaps["subject_id"], r=2)))
)
for lamap in topomaps["latent map"].values:
    for comb in combinations_with_replacement(topomaps["subject_id"].values, r=2):
        to_corr = topomaps.sel(
            {"subject_id": [comb[0], comb[1]], "latent map": lamap}
        ).dropna("channels")
        xrs.append(
            xr.DataArray(
                corr2_coeff(to_corr[:, 0, :].values.T, to_corr[:, 1, :].values.T),
                dims=["type1", "type2"],
                coords={
                    "type1": topomaps["type"].values,
                    "type2": topomaps["type"].values,
                },
            )
            .assign_coords({"latent map": lamap, "subj1": comb[0], "subj2": comb[1]})
            .expand_dims(["latent map", "subj1", "subj2"])
        )
        pbar.update(1)



  2%|██▉                                                                                                                                                              | 1487/82824 [00:11<10:10, 133.22it/s]


  0%|                                                                                                                                                                   | 48/82824 [00:00<02:53, 476.96it/s]

  0%|▏                                                                                                                                                                 | 104/82824 [00:00<02:37, 524.37it/s]

  0%|▎                                                                                                                                                                 | 161/82824 [00:00<02:32, 541.45it/s]

  0%|▍                                                                                                                                                                 | 217/

In [ ]:
xrs = xr.combine_by_coords(xrs)



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 82824/82824 [03:13<00:00, 395.65it/s]

In [ ]:
xrs